<a href="https://colab.research.google.com/github/donghuna/AI_Expert_Lecture_Files/blob/main/Lab2_2_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

from torchvision.datasets import MNIST
from torchvision.transforms import Compose,ToTensor

from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import PIL

#### Connect to local google drive & settings for export/import models

In [ ]:
# drive.mount('/content/gdrive')
# !mkdir ./gdrive/'My Drive'/MNIST_models/

#### DNN model

In [2]:
class MNISTDNN(nn.Module):
    def __init__(self,IMG_SIZE=28):
        super(MNISTDNN,self).__init__()
        self.fc1 = nn.Linear(IMG_SIZE*IMG_SIZE,32)
        self.BN1 = torch.nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32,10)

    def forward(self,x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.BN1(x)
        x = self.fc2(x)
        x = torch.softmax(x,dim=-1)
        return x

#### CNN model

In [3]:
class MNISTCNN(nn.Module):
    def __init__(self,IMG_SIZE=28):
        super(MNISTCNN,self).__init__()
        self.conv1 = nn.Conv2d(1,8,5,stride=2) # in_channels: int, out_channels: int, kernel_size: _size_2_t
        self.BN1 = torch.nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8,8,5,stride=2)
        self.BN2 = torch.nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(8,8,3,stride=1)
        self.fc = nn.Linear(8*2*2,10)

    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.BN1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.BN2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = x.view(-1,8*2*2)
        x = self.fc(x)
        x = torch.softmax(x,dim=-1)
        return x

#### Util function for calculating accuracy

In [4]:
def compute_acc(argmax,y):
    count = 0
    for i in range(len(argmax)):
        if argmax[i]==y[i]:
            count+=1
    return count / len(argmax)

#### hyperparameters & datasets

In [5]:
IMG_SIZE = 28
BATCH_SIZE = 256
LEARNING_RATE = 0.001
NUM_EPOCHES = 30

In [6]:
transforms = Compose([
    ToTensor(),
])

# trainset = MNIST('/content/gdrive/My Drive/MNIST_models/',train=True,transform=transforms,download=True)
# testset = MNIST('/content/gdrive/My Drive/MNIST_models/',train=False,transform=transforms,download=True)

# download path 정의
download_root = './MNIST_DATASET'

trainset = MNIST(download_root, transform=transforms, train=True, download=True)
testset = MNIST(download_root, transform=transforms, train=False, download=True)


args = {
    'num_workers' : 1,
    'batch_size' : BATCH_SIZE,
    'shuffle' : True,
}

train_loader = DataLoader(trainset,**args)
test_loader = DataLoader(testset,**args)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:01<00:00, 5105132.78it/s]


Extracting ./MNIST_DATASET/MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST_DATASET/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 129789.26it/s]


Extracting ./MNIST_DATASET/MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST_DATASET/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1271055.90it/s]


Extracting ./MNIST_DATASET/MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST_DATASET/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 12219710.56it/s]

Extracting ./MNIST_DATASET/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST_DATASET/MNIST/raw



In [13]:
!ls ./MNIST_DATASET/MNIST

raw


####Training part(DNN)

In [ ]:
model = MNISTDNN(IMG_SIZE).cuda()

# Model의 parameters 수를 확인하기 위한 코드
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
num_params = sum([np.prod(p.size()) for p in model_parameters])
print("number of parameters : {}".format(num_params))

optimizer = Adam(model.parameters(),lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHES):
    tot_loss = 0.0

    for x,y in train_loader:
        optimizer.zero_grad()
        x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
        y_ = model(x) # y_ predict
        loss = loss_fn(y_, y.cuda()) # loss구할때는 왜 y_predict와 y를 바로 비교하고, compute_acc에서는 torch.max값을 구해서 비교??
        loss.backward()
        tot_loss+=loss.item()
        optimizer.step()

    print("Epoch {}, Loss(train) : {}".format(epoch+1,tot_loss))
    if epoch % 2 == 1:ㅁ
        x,y = next(iter(test_loader))
        x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
        y_ = model(x)
        aa, argmax = torch.max(y_,dim=-1) # dim -1 의미는?
        print(aa, aa.shape)
        print(argmax, argmax.shape)
        test_acc = compute_acc(argmax, y.numpy())

        print("Acc(val) : {}".format(test_acc))

# torch.save(model.state_dict(), "/content/gdrive/My Drive/MNIST_models/DNN.pt")

y값을 확인하기 위한 출력용 코드

In [27]:
model = MNISTDNN(IMG_SIZE).cuda()

# Model의 parameters 수를 확인하기 위한 코드
# model_parameters = filter(lambda p: p.requires_grad, model.parameters())
# num_params = sum([np.prod(p.size()) for p in model_parameters])
# print("number of parameters : {}".format(num_params))

optimizer = Adam(model.parameters(),lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHES):
    tot_loss = 0.0

    for x,y in train_loader:
        optimizer.zero_grad()
        x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
        y_ = model(x) # y_ predict
        # print(y_, y_.shape)
        # print(y, y.shape)
        loss = loss_fn(y_, y.cuda()) # loss구할때는 왜 y_predict와 y를 바로 비교하고, compute_acc에서는 torch.max값을 구해서 비교??
        loss.backward()
        tot_loss+=loss.item()
        optimizer.step()
        break

    print("Epoch {}, Loss(train) : {}".format(epoch+1,tot_loss))
    # if epoch % 2 == 1:
    if 1 == 1:
        x,y = next(iter(test_loader))
        x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
        y_ = model(x)
        aa, argmax = torch.max(y_,dim=-1) # dim -1 의미는 dim의 제일 높은 단계중에서 max값을 찾겠다는 의미, dim을 설정하게 되면, value와 index가 함께 출력되는 특징이 있습니다.
        print(aa, aa.shape)
        print(argmax, argmax.shape)
        test_acc = compute_acc(argmax, y.numpy())

        print("Acc(val) : {}".format(test_acc))
    break

# torch.save(model.state_dict(), "/content/gdrive/My Drive/MNIST_models/DNN.pt")

Epoch 1, Loss(train) : 2.3047361373901367
tensor([0.2291, 0.2193, 0.1804, 0.1519, 0.2822, 0.1710, 0.2145, 0.1902, 0.2404,
        0.2320, 0.2796, 0.2770, 0.3548, 0.4426, 0.2092, 0.2203, 0.1568, 0.1582,
        0.1562, 0.2093, 0.1969, 0.3433, 0.3172, 0.2734, 0.2057, 0.2203, 0.1432,
        0.2066, 0.3195, 0.2981, 0.1833, 0.1645, 0.1701, 0.2016, 0.1940, 0.1944,
        0.1821, 0.3117, 0.1470, 0.2811, 0.1907, 0.2911, 0.1814, 0.1846, 0.2087,
        0.1669, 0.1661, 0.1725, 0.1616, 0.2854, 0.2181, 0.2451, 0.2340, 0.1738,
        0.1414, 0.2429, 0.2051, 0.1607, 0.1721, 0.1859, 0.4302, 0.2211, 0.3013,
        0.3439, 0.2063, 0.2254, 0.1890, 0.1937, 0.2640, 0.2252, 0.2268, 0.1668,
        0.1976, 0.2842, 0.3021, 0.1646, 0.1776, 0.1479, 0.2237, 0.4347, 0.1738,
        0.2938, 0.1456, 0.1989, 0.2070, 0.2602, 0.1465, 0.1903, 0.2460, 0.4224,
        0.1310, 0.1757, 0.2356, 0.3198, 0.2994, 0.2617, 0.1563, 0.2189, 0.1976,
        0.1706, 0.1794, 0.3872, 0.2498, 0.1849, 0.1462, 0.2162, 0.2910, 0.2176

CUDA 사용하지 않고 동작하는 코드

In [23]:
# model = MNISTDNN(IMG_SIZE).cuda()
model = MNISTDNN(IMG_SIZE)

# Model의 parameters 수를 확인하기 위한 코드
# model_parameters = filter(lambda p: p.requires_grad, model.parameters())
# num_params = sum([np.prod(p.size()) for p in model_parameters])
# print("number of parameters : {}".format(num_params))

optimizer = Adam(model.parameters(),lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHES):
    tot_loss = 0.0

    for x,y in train_loader:
        optimizer.zero_grad()
        # x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
        x = x.view(-1,IMG_SIZE*IMG_SIZE) # view를 정의하는 이유는 model에서 self.fc1 = nn.Linear(IMG_SIZE*IMG_SIZE,32)의 input 형식에 맞춰주는것임
        y_ = model(x)
        loss = loss_fn(y_, y)
        loss.backward()
        tot_loss+=loss.item()
        optimizer.step()

    print("Epoch {}, Loss(train) : {}".format(epoch+1,tot_loss))
    if epoch % 2 == 1:
        x,y = next(iter(test_loader))
        x = x.view(-1,IMG_SIZE*IMG_SIZE)
        y_ = model(x)
        _, argmax = torch.max(y_,dim=-1)
        test_acc = compute_acc(argmax,y.numpy())

        print("Acc(val) : {}".format(test_acc))

# torch.save(model.state_dict(), "/content/gdrive/My Drive/MNIST_models/DNN.pt")

Epoch 1, Loss(train) : 405.0010612010956
Epoch 2, Loss(train) : 367.5182956457138
Acc(val) : 0.95703125
Epoch 3, Loss(train) : 360.7984906435013
Epoch 4, Loss(train) : 357.62138509750366
Acc(val) : 0.90234375
Epoch 5, Loss(train) : 355.5156397819519
Epoch 6, Loss(train) : 354.0269241333008
Acc(val) : 0.94921875
Epoch 7, Loss(train) : 352.9148017168045
Epoch 8, Loss(train) : 352.2267256975174
Acc(val) : 0.953125
Epoch 9, Loss(train) : 351.45687234401703
Epoch 10, Loss(train) : 350.8423923254013
Acc(val) : 0.9765625
Epoch 11, Loss(train) : 350.50638234615326
Epoch 12, Loss(train) : 349.9335335493088
Acc(val) : 0.9765625
Epoch 13, Loss(train) : 349.6529446840286
Epoch 14, Loss(train) : 349.2466480731964
Acc(val) : 0.9765625
Epoch 15, Loss(train) : 348.948228597641
Epoch 16, Loss(train) : 348.8442051410675
Acc(val) : 0.953125
Epoch 17, Loss(train) : 348.59975612163544
Epoch 18, Loss(train) : 348.3664346933365
Acc(val) : 0.94921875
Epoch 19, Loss(train) : 348.17678582668304
Epoch 20, Loss(t

KeyboardInterrupt: 

In [16]:
torch.save(model.state_dict(), "./DNN.pt")

In [17]:
model_test = MNISTDNN(IMG_SIZE).cuda()
model_test.load_state_dict(torch.load("./DNN.pt"))
model_test.eval()
x,y = next(iter(test_loader))
x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
y_ = model_test(x)
_, argmax = torch.max(y_,dim=-1)
test_acc = compute_acc(argmax,y.numpy())

print("Acc(test) : {}".format(test_acc))

Acc(test) : 0.97265625


#### Training part(CNN)

In [18]:
model = MNISTCNN(IMG_SIZE).cuda()

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
# model.parameters() : list 형태의 파라미터
num_params = sum([np.prod(p.size()) for p in model_parameters])
# num_params : 모델 파라미터 수
print("number of parameters : {}".format(num_params))

optimizer = Adam(model.parameters(),lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHES):
    tot_loss = 0.0

    for x,y in train_loader:
        optimizer.zero_grad()
        x = x.cuda()
        y_ = model(x)
        loss = loss_fn(y_, y.cuda())
        loss.backward()
        tot_loss+=loss.item()
        optimizer.step()

    print("Epoch {}, Loss(train) : {}".format(epoch+1,tot_loss))
    if epoch % 2 == 1:
        model.eval()

        x,y = next(iter(test_loader))
        x = x.cuda()
        y_ = model(x)
        _, argmax = torch.max(y_,dim=-1)
        test_acc = compute_acc(argmax,y.numpy())

        print("Acc(test) : {}".format(test_acc))

        model.train()

torch.save(model.state_dict(), "./CNN.pt")

number of parameters : 2762
Epoch 1, Loss(train) : 434.5468850135803
Epoch 2, Loss(train) : 361.9365322589874
Acc(test) : 0.96484375
Epoch 3, Loss(train) : 355.2590903043747
Epoch 4, Loss(train) : 352.79480969905853
Acc(test) : 0.9609375
Epoch 5, Loss(train) : 351.18767166137695
Epoch 6, Loss(train) : 350.317409992218
Acc(test) : 0.98046875
Epoch 7, Loss(train) : 349.603174328804
Epoch 8, Loss(train) : 349.0490720272064
Acc(test) : 0.98046875
Epoch 9, Loss(train) : 348.68840992450714
Epoch 10, Loss(train) : 348.3464721441269
Acc(test) : 0.984375
Epoch 11, Loss(train) : 348.0965973138809
Epoch 12, Loss(train) : 347.8390871286392
Acc(test) : 0.984375
Epoch 13, Loss(train) : 347.7000914812088
Epoch 14, Loss(train) : 347.5057452917099
Acc(test) : 0.98046875
Epoch 15, Loss(train) : 347.3235617876053
Epoch 16, Loss(train) : 347.1446521282196
Acc(test) : 0.97265625
Epoch 17, Loss(train) : 347.08527052402496
Epoch 18, Loss(train) : 346.93141782283783
Acc(test) : 0.96875
Epoch 19, Loss(train) :

In [ ]:
model_test = MNISTCNN(IMG_SIZE).cuda()
model_test.load_state_dict(torch.load("./CNN.pt"))
model_test.eval()
x,y = next(iter(test_loader))
x = x.cuda()
y_ = model_test(x)
_, argmax = torch.max(y_,dim=-1)
test_acc = compute_acc(argmax,y.numpy())

print("Acc(test) : {}".format(test_acc))